In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [17]:
db_tweets = pd.read_csv('../data/db_annotated_tweets.csv', index_col=False)
db_tweets = db_tweets.fillna('')
db_tweets['UserName'] = db_tweets['UserName'].apply(lambda x: x.replace('@', ''))

C:\Users\gokce\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (0,1,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
db_tweets['MentionedUser'] = db_tweets['mentions'].apply(lambda x: [i.split(',')[0] for i in x.split(';')])
mentions = db_tweets.explode('MentionedUser')[['UserName', 'MentionedUser']]
mentions = mentions[(mentions['UserName'] != '') & (mentions['MentionedUser'] != '') & (mentions['MentionedUser'] != mentions['UserName'])]
mention_counts = mentions.groupby(['UserName', 'MentionedUser'])['UserName'].count().to_frame().rename(columns={'UserName':'count'}).reset_index()
mention_counts

,UserName,MentionedUser,count
0,) 1766,816AJ18,1
1,) 1766,russiancosmist,1
2,000Dillon000,CBedfordDC,1
3,001Prometheus,Guns_Velez,1
4,001Prometheus,NoNutNoSlut,1
...,...,...,...
229946,zzzdogman,eveforamerica,1
229947,zzzdogman,rezazia,1
229948,zzzjamieg,RadioFreeTom,1
229949,zzzuperior,cityafreaks,1


In [53]:
mention_counts['weights'] = mention_counts.groupby('UserName')['count'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
mention_counts = mention_counts.fillna(1.0)
mention_counts.sort_values(by='weights', ascending=False)

,UserName,MentionedUser,count,weights
0,) 1766,816AJ18,1,1.0
148413,bravoloverfan,vicsepulveda,1,1.0
148456,breadboi2077,DonutOperator,1,1.0
148457,breadboosy,johnpavlovitz,1,1.0
148458,breadman222,lavern_spicer,1,1.0
...,...,...,...,...
22802,CalumetCoRepub,RepGwenMoore,1,0.0
22803,CalumetCoRepub,RichieMcGinniss,1,0.0
22804,CalumetCoRepub,Shinedown1911,1,0.0
22805,CalumetCoRepub,elaadeliahu,1,0.0


In [57]:
retweets = db_tweets[db_tweets['retweet'] != '']
retweet_counts = retweets.groupby(['UserName', 'retweet'])['UserName'].count().to_frame().rename(columns={'UserName':'count'}).reset_index()
retweet_counts

,UserName,retweet,count
0,000Dillon000,CBedfordDC,1
1,0045James,imUrB00gieman,1
2,0071Holly,fake_biden,1
3,008moonside,DougCameron51,1
4,00Lilac_Sky00,WalterDavis4L,1
...,...,...,...
112539,zzz_das,FAsintomatico,1
112540,zzzdogman,McKaylaRoseJ,1
112541,zzzdogman,eveforamerica,1
112542,zzzdogman,rezazia,1


In [59]:
retweet_counts['count'].value_counts()

1     108971
2       2906
3        418
4        134
5         51
6         23
7         13
8         10
9          7
12         4
11         2
67         1
14         1
16         1
19         1
21         1
Name: count, dtype: int64

In [60]:
retweet_counts['weights'] = retweet_counts.groupby('UserName')['count'].apply(lambda x: (x-min(x))/(max(x)-min(x)))
retweet_counts

,UserName,retweet,count,weights
0,000Dillon000,CBedfordDC,1,NaN
1,0045James,imUrB00gieman,1,NaN
2,0071Holly,fake_biden,1,NaN
3,008moonside,DougCameron51,1,NaN
4,00Lilac_Sky00,WalterDavis4L,1,NaN
...,...,...,...,...
112539,zzz_das,FAsintomatico,1,NaN
112540,zzzdogman,McKaylaRoseJ,1,NaN
112541,zzzdogman,eveforamerica,1,NaN
112542,zzzdogman,rezazia,1,NaN


In [61]:
retweet_counts['weights'].value_counts()

0.000000    7336
1.000000    2290
0.500000      96
0.333333      28
0.250000      18
0.142857      17
0.090909      12
0.166667      11
0.666667       8
0.285714       8
0.125000       6
0.750000       5
0.571429       3
0.066667       3
0.428571       3
0.200000       2
0.714286       2
0.181818       2
0.272727       2
0.375000       1
0.833333       1
0.800000       1
0.133333       1
Name: weights, dtype: int64

In [63]:
retweet_counts.fillna(1.0)

,UserName,retweet,count,weights
0,000Dillon000,CBedfordDC,1,1.0
1,0045James,imUrB00gieman,1,1.0
2,0071Holly,fake_biden,1,1.0
3,008moonside,DougCameron51,1,1.0
4,00Lilac_Sky00,WalterDavis4L,1,1.0
...,...,...,...,...
112539,zzz_das,FAsintomatico,1,1.0
112540,zzzdogman,McKaylaRoseJ,1,1.0
112541,zzzdogman,eveforamerica,1,1.0
112542,zzzdogman,rezazia,1,1.0


In [66]:
interactions = pd.merge(mention_counts, retweet_counts, left_on=['UserName', 'MentionedUser'], right_on=['UserName', 'retweet'], how='outer', suffixes=('_mention', '_rt'))
interactions = interactions.fillna(0)

In [67]:
interactions.to_csv('../data/interactions.csv')